# Creatring BIDS dataset

## Creating a dataset description

In [1]:
addpath(genpath(fullfile(pwd, '..', 'src')));

cfg.dir.output = fullfile(pwd, 'output');

cfg.task.name = 'test task';

cfg.subject.subjectNb = 1;
cfg.subject.runNb = 1;

cfg.testingDevice = 'mri';

cfg.eyetracker.do = 1;

cfg.verbose = false;

In [2]:
cfg.bids.datasetDescription.Name = 'my new bids data set';
cfg.bids.datasetDescription.BIDSVersion =  '1.3';
cfg.bids.datasetDescription.License = 'CC-BY';
cfg.bids.datasetDescription.Authors = {'John Wick', 'Napoléon Bonaparte'};
cfg.bids.datasetDescription.Acknowledgements = '';
cfg.bids.datasetDescription.HowToAcknowledge = '';
cfg.bids.datasetDescription.Funding = {''};
cfg.bids.datasetDescription.ReferencesAndLinks = {''};
cfg.bids.datasetDescription.DatasetDOI = '';

[cfg] = createFilename(cfg);

In [3]:
mkdir(fullfile(cfg.dir.outputSubject, cfg.fileName.modality));

createDatasetDescription(cfg)

In [4]:
!tree

.
├── basic_usage.ipynb
├── creating_BIDS_dataset.ipynb
├── more_on_saving.ipynb
├── output
│   └── source
│       ├── dataset_description.json
│       └── sub-001
│           └── ses-001
│               └── func
└── README.md

5 directories, 5 files



In [5]:
!cat output/source/dataset_description.json

{
    "Acknowledgements": "",
    "Authors": [
        "John Wick",
        "Napoléon Bonaparte"
    ],
    "BIDSVersion": "1.3",
    "DatasetDOI": "",
    "Funding": [""],
    "HowToAcknowledge": "",
    "License": "CC-BY",
    "Name": "my new bids data set",
    "ReferencesAndLinks": [""]
}


## fMRI

### fMRI specific name suffixes

You can specify some extra information for you fMRI runs as some of this information can be important for filenaming or to add to some JSON files.

In [6]:
cfg.mri.repetitionTime = 2.1;
cfg.mri.contrastEnhancement = [];
cfg.mri.phaseEncodingDirection = [];
cfg.mri.reconstruction = 'fast';
cfg.mri.echo = '1';
cfg.mri.acquisition = 'slow';

In [7]:
[cfg] = createFilename(cfg);

In [8]:
cfg.fileName.events

ans = sub-001_ses-001_task-testTask_acq-slow_rec-fast_run-001_echo-1_events_date-202008090023.tsv


### Saving a `_bold.json` file

There is bare minimum functionality to create a `_bold.json` file for fMRI runs: the information in there wil be that required to pass a BIDS validation but no more. You should REALLY get most of the information for `bold.json` from your favorite DICOM to Nifti converter

This function is there because it can help put in JSON format some of the information that would NOT be available to a converter (usually information about your task).

In [9]:
createBoldJson(cfg)

In [10]:
!tree

.
├── basic_usage.ipynb
├── creating_BIDS_dataset.ipynb
├── more_on_saving.ipynb
├── output
│   └── source
│       ├── dataset_description.json
│       └── sub-001
│           └── ses-001
│               └── func
│                   └── sub-001_ses-001_task-testTask_acq-slow_rec-fast_run-001_echo-1_bold_date-202008090023.json
└── README.md

5 directories, 6 files



In [11]:
!cat output/source/sub-001/ses-001/func/sub-001*bold_date-*.json

{
    "Instructions": "",
    "RepetitionTime": [],
    "SliceTiming": [],
    "TaskDescription": "",
    "TaskName": "test Task"
}


### Adding extra info to your `_bold.json`

One thing this function CAN do though is help you save some of the information about your experiment in a json format rather than a `.mat` format. The former being a simple text file you can easily open on any computer.

For example if you have defined some other parameters in another structure.

In [12]:
%% Timing info

% Time between blocs in secs
parameters.timing.IBI = 1.8 * 3; % 8;
% Time between events in secs
parameters.timing.ISI = 0.1;
% Number of seconds before the motion stimuli are presented
parameters.timing.onsetDelay = .1;
% Number of seconds after the end all the stimuli before ending the run
parameters.timing.endDelay = .1;

parameters.timing.eventDuration = 0.8; % second

%% Visual Stimulation

% Speed in visual angles / second
parameters.dot.speed = 30;
% Coherence Level (0-1)
parameters.dot.coherence = 1;
% Number of dots per visual angle square.
parameters.dot.density = .1;
% Dot life time in seconds
parameters.dot.lifeTime = 10;
% proportion of dots killed per frame
parameters.dot.proportionKilledPerFrame = 0;
% Dot Size (dot width) in visual angles.
parameters.dot.size = 1;

In [13]:
createBoldJson(cfg, parameters)

In [14]:
!cat output/source/sub-001/ses-001/func/sub-001*bold_date-*.json

{
    "Instructions": "",
    "RepetitionTime": [],
    "SliceTiming": [],
    "TaskDescription": "",
    "TaskName": "test Task",
    "timing": {
        "IBI": 5.4,
        "ISI": 0.1,
        "onsetDelay": 0.1,
        "endDelay": 0.1,
        "eventDuration": 0.8
    },
    "dot": {
        "speed": 30,
        "coherence": 1,
        "density": 0.1,
        "lifeTime": 10,
        "proportionKilledPerFrame": 0,
        "size": 1
    }
}


## Converting your data set from source to raw

Before we do that, let's adding some actual data to the dataset.

In [16]:

logFile.extraColumns = {'target_position', 'target_type'};

logFile = saveEventsFile('open', cfg, logFile);

logFile(1,1).onset = 2;
logFile(1,1).trial_type = 'motion_up';
logFile(1,1).duration = 1;
logFile(1,1).target_position = 'left';
logFile(1,1).target_type = 'audio';

logFile(2,1).onset = 6;
logFile(2,1).trial_type = 'static';
logFile(2,1).duration = 1.2;
logFile(2,1).target_position = 'right';
logFile(2,1).target_type = 'visual';

logFile = saveEventsFile('save', cfg, logFile);

saveEventsFile('close', cfg, logFile);

onset	duration	trial_type	target_position	target_type
2.000000	1.000000	motion_up	left	audio	
6.000000	1.200000	static	right	visual	


In [17]:
!tree

.
├── basic_usage.ipynb
├── creating_BIDS_dataset.ipynb
├── more_on_saving.ipynb
├── output
│   └── source
│       ├── dataset_description.json
│       └── sub-001
│           └── ses-001
│               └── func
│                   ├── sub-001_ses-001_task-testTask_acq-slow_rec-fast_run-001_echo-1_bold_date-202008090023.json
│                   ├── sub-001_ses-001_task-testTask_acq-slow_rec-fast_run-001_echo-1_events_date-202008090023.json
│                   └── sub-001_ses-001_task-testTask_acq-slow_rec-fast_run-001_echo-1_events_date-202008090023.tsv
└── README.md

5 directories, 8 files



The very basic conversion here mostly entails:
- adding empty README and CHANGES files in the `source` folder
- copying the content of the `source` folder into a `raw` folder
- removing the date suffixes
- compressing `_stim.tsv` into `_stim.tsv.gz`

In [18]:
convertSourceToRaw(cfg)

In [19]:
!tree

.
├── basic_usage.ipynb
├── creating_BIDS_dataset.ipynb
├── more_on_saving.ipynb
├── output
│   ├── rawdata
│   │   ├── CHANGES
│   │   ├── dataset_description.json
│   │   ├── README
│   │   └── sub-001
│   │       └── ses-001
│   │           └── func
│   │               ├── sub-001_ses-001_task-testTask_acq-slow_rec-fast_run-001_echo-1_bold.json
│   │               ├── sub-001_ses-001_task-testTask_acq-slow_rec-fast_run-001_echo-1_events.json
│   │               └── sub-001_ses-001_task-testTask_acq-slow_rec-fast_run-001_echo-1_events.tsv
│   └── source
│       ├── CHANGES
│       ├── dataset_description.json
│       ├── README
│       └── sub-001
│           └── ses-001
│               └── func
│                   ├── sub-001_ses-001_task-testTask_acq-slow_rec-fast_run-001_echo-1_bold_date-202008090023.json
│                   ├── sub-001_ses-001_task-testTask_acq-slow_rec-fast_run-001_echo-1_events_date-202008090023.json
│                   └── sub-001_ses-001_task-testTask_acq-slo